In [ ]:
CREATE OR REPLACE DATABASE PRESCRIPTIVE_MAINTANANCE;

CREATE OR REPLACE STAGE PUMP_DOCUMENTATION ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE');

CREATE OR REPLACE WAREHOUSE CORTEX_SEARCH_WH WITH WAREHOUSE_SIZE='MEDIUM';

COPY Prescriptive Maintanance Files to the PRESCRIPTIVE_MAINTANANCE.PUBLIC.PUMP_DOCUMENTATION Stage

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import numpy as np
import pandas as pd
import altair as alt

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
CREATE OR REPLACE TABLE PRESCRIPTIVE_MAINTANANCE.PUBLIC.PLM_DATA (
  EQUIPMENT_ID integer, 
  PUMP_TYPE VARCHAR,
  SERIAL_NUMBER VARCHAR,
  INSTALLATION_LOCATION VARCHAR,
  CUSTOMER_NAME VARCHAR);

INSERT INTO PRESCRIPTIVE_MAINTANANCE.PUBLIC.PLM_DATA 
   VALUES (
    1,
    'Centrifugal Pump',
    'PMP-123456',
    '123 Main Street, Anytown, USA', 
     'XYZ Manufacturing Inc.');

In [ ]:
# Constants
MINUTES_IN_TWO_DAYS = 1 * 24 * 60
NORMAL_VIBRATION_RANGE = (60, 63)
NORMAL_TEMPERATURE_RANGE = (90, 100)

# Generate time series data
time_series = pd.date_range(start='2025-01-01', periods=MINUTES_IN_TWO_DAYS, freq='min')

# Generate normal operation data
vibration = np.random.uniform(NORMAL_VIBRATION_RANGE[0], NORMAL_VIBRATION_RANGE[1], MINUTES_IN_TWO_DAYS)
temperature = np.random.uniform(NORMAL_TEMPERATURE_RANGE[0], NORMAL_TEMPERATURE_RANGE[1], MINUTES_IN_TWO_DAYS)

# Introduce gradual rise in vibration and temperature at the end of the second day
rise_start_index = int(MINUTES_IN_TWO_DAYS * 0.7)  # Start rise at 90% of the time series
temp_rise_start_index = int(MINUTES_IN_TWO_DAYS * 0.9)  # Start rise at 90% of the time series
rise_end_index = MINUTES_IN_TWO_DAYS

# Gradual rise in vibration
vibration[rise_start_index:rise_end_index] = np.linspace(
    vibration[rise_start_index],
    vibration[rise_start_index] + 10,
    rise_end_index - rise_start_index
) + np.random.normal(0, 1, rise_end_index - rise_start_index)

# Gradual rise in temperature
temperature[temp_rise_start_index:rise_end_index] = np.linspace(
    temperature[temp_rise_start_index],
    temperature[temp_rise_start_index] + 6,
    rise_end_index - temp_rise_start_index
) + np.random.normal(0, 1, rise_end_index - temp_rise_start_index)

# Create DataFrame
data = pd.DataFrame({
    'Time': time_series,
    'Vibration': vibration,
    'Temperature': temperature
})

a = alt.Chart(data).mark_line(opacity=1, color='blue').encode(
    x='Time', y='Vibration')

b = alt.Chart(data).mark_line(opacity=1, color='red').encode(
    x='Time', y='Temperature')

c = alt.layer(a,b)

st.header("Vibration value from the process")
st.altair_chart(c, use_container_width=True)

# Plot the data

#font = {'family' : 'normal',
#        'weight' : 'normal',
#        'size'   : 6}
#
#plt.rc('font', **font)
#
#plt.figure(figsize=(6, 3))
#plt.plot(data['Time'], data['Vibration'], label='Vibration')
#plt.plot(data['Time'], data['Temperature'], label='Temperature')
#plt.xlabel('Time')
#plt.ylabel('Values')
#plt.title('Pump Vibration and Temperature')
#plt.legend()
#plt.show()

In [ ]:
session.write_pandas(data, "IOT_DATA", auto_create_table=True, use_logical_type=True, overwrite=True)

In [ ]:
CREATE OR REPLACE VIEW PRESCRIPTIVE_MAINTANANCE.PUBLIC.TRAINING_DATA
AS
SELECT "Time" AS TIME, "Vibration" AS VIBRATION FROM IOT_DATA WHERE "Time" <= '2025-01-01 03:00:00';

CREATE OR REPLACE VIEW PRESCRIPTIVE_MAINTANANCE.PUBLIC.INFERENCE_DATA
AS
SELECT "Time" AS TIME, "Vibration" AS VIBRATION FROM IOT_DATA WHERE "Time" > '2025-01-01 03:00:00';

ALTER WAREHOUSE DEV SET warehouse_size = MEDIUM;

![text](https://quickstarts.snowflake.com/guide/deploying_custom_models_to_snowflake_model_registry/img/cdfcb92c6d412c69.png)

In [ ]:
CREATE OR REPLACE SNOWFLAKE.ML.ANOMALY_DETECTION vibration_anomaly_prescriptive_maintanance(
  INPUT_DATA => TABLE(PRESCRIPTIVE_MAINTANANCE.PUBLIC.TRAINING_DATA),
  TIMESTAMP_COLNAME => 'TIME',
  TARGET_COLNAME => 'VIBRATION',
  LABEL_COLNAME => '');

In [ ]:
CREATE OR REPLACE TABLE PRESCRIPTIVE_MAINTANANCE.PUBLIC.VIBRATION_ANOMALY AS
  SELECT 1 as EQUIPMENT_ID, * FROM TABLE(vibration_anomaly_prescriptive_maintanance!DETECT_ANOMALIES(
    INPUT_DATA => TABLE(PRESCRIPTIVE_MAINTANANCE.PUBLIC.INFERENCE_DATA),
    TIMESTAMP_COLNAME => 'TIME',
    TARGET_COLNAME => 'VIBRATION'
  ))

In [ ]:
SELECT * FROM PRESCRIPTIVE_MAINTANANCE.PUBLIC.VIBRATION_ANOMALY WHERE IS_ANOMALY = True;

In [ ]:
dataframe = session.sql("WITH x as (SELECT TS, IS_ANOMALY FROM PRESCRIPTIVE_MAINTANANCE.PUBLIC.VIBRATION_ANOMALY WHERE IS_ANOMALY = TRUE and PERCENTILE > 0.5) select TIME, VIBRATION, IFF(IS_ANOMALY=True, VIBRATION, null) as IS_ANOMALY from PRESCRIPTIVE_MAINTANANCE.PUBLIC.INFERENCE_DATA iot LEFT OUTER JOIN x anomalies on iot.TIME = anomalies.TS;")
df = dataframe.to_pandas()

a = alt.Chart(df).mark_line(opacity=1).encode(
    x='TIME', y='VIBRATION')

b = alt.Chart(df).mark_circle(opacity=1, color="red").encode(
    x='TIME', y='IS_ANOMALY')

c = alt.layer(a, b)

st.header("Vibration Anomalies")
st.altair_chart(c, use_container_width=True)

In [ ]:
CREATE OR REPLACE TABLE PARSED_TEXT
AS
WITH pdf_files AS (
    SELECT DISTINCT METADATA$FILENAME AS relative_path
    FROM @PRESCRIPTIVE_MAINTANANCE.PUBLIC.PUMP_DOCUMENTATION
    WHERE METADATA$FILENAME ILIKE '%.pdf'
)
SELECT
    pdf_files.relative_path,
    SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
    '@PRESCRIPTIVE_MAINTANANCE.PUBLIC.PUMP_DOCUMENTATION',
    pdf_files.relative_path,
    {'mode': 'LAYOUT'}
    ):content AS raw_text
FROM pdf_files;

In [ ]:
CREATE OR REPLACE TABLE CHUNKED_TEXT
AS
SELECT
   relative_path,
   c.value::varchar as chunked_text
FROM
   PRESCRIPTIVE_MAINTANANCE.PUBLIC.PARSED_TEXT,
   LATERAL FLATTEN( input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
      raw_text,
      'markdown',
      1000,
      100
   )) c;

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE PRESCRIPTIVE_MAINTANANCE.PUBLIC.PUMP_DOCUMENTATION_SERVICE
  ON chunked_text
  WAREHOUSE = cortex_search_wh
  TARGET_LAG = '1 day'
  AS (
    SELECT
        chunked_text
    FROM PRESCRIPTIVE_MAINTANANCE.PUBLIC.CHUNKED_TEXT
);

![text](https://miro.medium.com/v2/resize:fit:720/format:webp/1*g7-pYioCgo8GIDEFstLlNA.png)

In [ ]:
ALTER WAREHOUSE DEV SET warehouse_size = XSMALL;

SELECT 
  SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
      'PRESCRIPTIVE_MAINTANANCE.PUBLIC.PUMP_DOCUMENTATION_SERVICE',
      '{
        "query": "Descripbe the customer and pump and How to fix the vibration problem for the anomaly",
        "columns":["chunked_text"],
        "limit":1
      }'
  );

In [ ]:
from snowflake.core import Root
root = Root(session)

# fetch service
my_service = (root
  .databases["PRESCRIPTIVE_MAINTANANCE"]
  .schemas["PUBLIC"]
  .cortex_search_services["PUMP_DOCUMENTATION_SERVICE"]
)

# query service for instructions how to fix vibration problem
repair_data = my_service.search(
  query="how to fix the vibration problem",
  columns=["chunked_text"],
  limit=5
)

# query service for pump technical spec
tech_data = my_service.search(
  query="centrifugal pump technical specifications",
  columns=["chunked_text"],
  limit=1
)

In [ ]:
anomaly_df = session.sql("with x as (SELECT TS as Timestamp, Y as Vibration_value, FORECAST, LOWER_BOUND, UPPER_BOUND, DISTANCE FROM PRESCRIPTIVE_MAINTANANCE.PUBLIC.VIBRATION_ANOMALY WHERE IS_ANOMALY = TRUE LIMIT 1) select object_construct(*)::varchar as anomaly_data From x").collect()

In [ ]:
plm_df = session.sql("with x as (SELECT * FROM PRESCRIPTIVE_MAINTANANCE.PUBLIC.PLM_DATA) select object_construct(*)::varchar as anomaly_data From x").collect()

In [ ]:
instructions = "You are a mechanical engineer in energy company. Use following data from the installation location, service reports, pump technical specifications, and from anomaly information to generate a detailed report for service engineer about the pump, its technical specifications, anomaly and what are the procedures to repair and test the pump."

In [ ]:
prompt = str(plm_df) + instructions + repair_data.to_json() + tech_data.to_json() + str(anomaly_df)
model = 'claude-3-5-sonnet'

In [ ]:
maint_recommendation = session.sql("SELECT snowflake.cortex.complete(?,?)", (model,prompt)).collect()[0][0]
print(maint_recommendation)